# 러닝 텐서플로우(Learning Tensorflow)
구글에서 만든 오픈소스 프레임워크

- 텐서 : 딥러닝에서 데이터를 표현하는 일반적인 방법
 + 다차원 배열
 + R,G,B (3차원 데이터)
 
 
- 텐서플로우 : 텐서 + 플로우

In [1]:
import tensorflow as tf
print(tf.__version__)

1.10.0


In [2]:
h = tf.constant("Hello")
w = tf.constant(" World!")
hw = h + w

with tf.Session() as sess: # sess = tf.Session()
    ans = sess.run(hw) #출력 가능
    
print(ans)

b'Hello World!'


# 소프트 맥스 회귀
- softmax : 확률
 + 전부 합치면 1
 + x -> y(0~1)

In [24]:
from tensorflow.examples.tutorials.mnist import input_data

DATA_DIR = '/tmp/data'
NUM_STEPS = 1000
MINIBATCH_SIZE = 100
learning_rate = 0.1

data = input_data.read_data_sets(DATA_DIR,one_hot=True)

x = tf.placeholder(tf.float32,[None,784])
W = tf.Variable(tf.zeros([784,10]))

y_true = tf.placeholder(tf.float32,[None,10])
y_pred = tf.matmul(x,W)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))

gd_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

correct_mask = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(correct_mask,tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for _ in range(10):
    for _ in range(NUM_STEPS):
        batch_xs,batch_ys = data.train.next_batch(MINIBATCH_SIZE)
        sess.run(gd_step,feed_dict={x:batch_xs, y_true:batch_ys})

    rst = sess.run(accuracy,feed_dict={x:data.test.images,y_true:data.test.labels})

    print("Accuracy: {:.4}%".format(rst*100))

Extracting /tmp/data\train-images-idx3-ubyte.gz
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz
Accuracy: 90.82%
Accuracy: 91.56%
Accuracy: 91.82%
Accuracy: 91.82%
Accuracy: 92.04%
Accuracy: 92.08%
Accuracy: 92.05%
Accuracy: 92.22%
Accuracy: 92.04%
Accuracy: 92.16%


# Foundation
## 연산 그래프

In [20]:
import tensorflow as tf
a = tf.constant(5)
b = tf.constant(2)
c = tf.constant(3)

d = tf.multiply(a,b)
e = tf.add(c,b)
f = tf.subtract(d,e)

sess = tf.Session()
out = sess.run(f)
sess.close()
print("out = {0}".format(out))

out = 5


## 그래프 그리기

In [28]:
g1 = tf.get_default_graph()
g2 = tf.Graph()

print(g1 is tf.get_default_graph())

with g2.as_default():
    print(g1 is tf.get_default_graph())

print(g1 is tf.get_default_graph())

True
False
True


## 페치
- 노드 리스트

In [31]:
with tf.Session() as sess:
    feches = [a,b,c,d,e,f]
    outs = sess.run(feches)
    
print("outs = {}".format(outs))
print(type(outs[0]))

outs = [5, 2, 3, 10, 5, 5]
<class 'numpy.int32'>


In [43]:
c = tf.constant(3.0)
print(c)

c = tf.constant(3.0,dtype=tf.float64)
print(c)

Tensor("Const_50:0", shape=(), dtype=float32)
Tensor("Const_51:0", shape=(), dtype=float64)


## CAST
- 형변환

In [46]:
x = tf.constant([1,2,3],name = 'x',dtype = tf.float32)
print(x.dtype)
x = tf.cast(x,tf.int32) # 데이터 타입 새로 적용
print(x.dtype)

<dtype: 'float32'>
<dtype: 'int32'>


# 텐서
- 1*1 = 스칼라
- 1*N = 벡터
- N*N = 행렬
- N*N*N = 3차원 배열

In [66]:
import numpy as np
c = tf.constant([[1,2,3],[4,5,6]])
print(c.shape)
c = tf.constant(np.array([ 
                        [[1,2,3],
                         [4,5,6]] , 
                        [[1,1,1],
                         [2,2,2]] 
                        ]))
print(c.shape)

(2, 3)
(2, 2, 3)


In [67]:
sess = tf.InteractiveSession()
c = tf.linspace(0.0,4.0,5)
print(c.eval())
sess.close()

[0. 1. 2. 3. 4.]


In [68]:
A = tf.constant([[1,2,3],[4,5,6]])
print(A.shape)

x = tf.constant([1,0,1])
print(x.shape)

(2, 3)
(3,)


In [69]:
#행렬을 곱하려면 x를 변환시켜야한다
x = tf.expand_dims(x,1)
print(x.get_shape())

b = tf.matmul(A,x)

sess = tf.InteractiveSession()
print(b.eval())
sess.close()

(3, 1)
[[ 4]
 [10]]


## 이름
```
tf.constant(4,dtype = tf.float64,name='c')
```

In [123]:
with tf.Graph().as_default():
    c1 = tf.constant(4,dtype = tf.float64,name='c')
    c2 = tf.constant(4,dtype = tf.int32,name='c')
    c3 = tf.constant(4,dtype = tf.float64,name='c')
    c4 = tf.constant(4,dtype = tf.int32,name='c')
    
print(c1.name)
print(c2.name)
print(c3.name)
print(c4.name)

c:0
c_1:0
c_2:0
c_3:0


In [132]:
g = tf.Graph()
with g.as_default(): # as_default를 사용하면 해당 그래프가 기본 그래프인 콘텍스트 할당
    c1 = tf.constant(4,dtype=tf.float64,name='c')
    with tf.name_scope("prefix_name"): # 그룹화
        c2 = tf.constant(4,dtype=tf.int32,name='c')
        c3 = tf.constant(4,dtype=tf.float64,name='c')
print(c1.name)
print(c2.name)
print(c3.name)

c:0
prefix_name/c:0
prefix_name/c_1:0


## 변수
```
tf.Variable()
```
- 변하는 수
- 고정된 상태를 유지 할 수 있다. -> 변수의 현재 상태가 반복 과정 속에서의 상태에 영향을 줄 수 있기 때문이다.
- 모델의 학습과정에서 사용

In [91]:
init_val = tf.random_normal((1,5),0,1)
sess = tf.Session()
print(sess.run(init_val))
var = tf.Variable(init_val, name='var') # 변수
print(var)

init = tf.global_variables_initializer() # 초기화 연산
sess.run(init)

print(sess.run(var))
sess.close()

[[ 2.1896572  -0.4299722  -0.46761578 -0.02529344 -1.7057981 ]]
<tf.Variable 'var_18:0' shape=(1, 5) dtype=float32_ref>
[[-0.00754655 -1.7255454   0.9521939  -0.2814501   0.97002816]]


In [147]:
# 변수 초기화
a = tf.Variable(5, name='my_variable_a')
 
a = a.assign_add(a)
 
init = tf.initialize_all_variables()
 
sess = tf.Session()
 
sess.run(init)
 
print(sess.run(a))
print(sess.run(a)) #변함
print(sess.run(a)) #변함

10
20
40


## 플레이스 홀더
```
tf.placehloder()
```
- 비어있는 변수
- 입력데이터를 밀어넣는다.
- 연산 및 입력에 사용

In [111]:
x_data = np.random.randn(5,10)
y_data = np.random.randn(10,1)

with tf.Graph().as_default():
    x = tf.placeholder(tf.float32,shape=(5,10)) #5*10 공간
    w = tf.placeholder(tf.float32,shape=(10,1)) #10*1 공간
    b = tf.fill((5,1),-1.) #5*1 공간 -1 초기화
    xw = tf.matmul(x,w)
    xwb = xw + b
    
    s = tf.reduce_max(xwb) #최대값으로 줄인다(reduce)
    with tf.Session() as sess:
        out = sess.run(s,feed_dict={x:x_data,w:y_data})
    print("out = {}".format(out))

out = 0.5401109457015991


## 최적화

> x = tf.placeholder(tf.float32,shape=[None,3]) <br>
> y_true = tf.placeholder(tf.float32,shape=None) <br>
> w = tf.Variable([[0,0,0]],dtype=tf.float32,name='weights') <br>
> b = tf.Variable(0,dtype=tf.float32,name='bias')

> y_pred = tf.matmul(w,tf.transpose(x)) + b 
- transpose = 전치함수

### 손실함수 정의
- 손실함수를 줄이는 것이 **우리의 목적**이다.
- MSE(평균제곱오차)
    + -> 실제값과 예측값 사이의 차이 제곱

> loss = tf.reduce_mean(tf.square(y_true-y_pred))

- **교차 엔트로피**(cross_entropy)
    + -> 유사성

> loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true,logits=y_pred) <br>
> loss = tf.reduce_mean(loss)

### 경사 하강법
- 최솟값 수렴

> optimizer = tf.train.GradientDescentOptimizer(learning_rate) <br>
> train = optimizer.minimize(loss)

In [114]:
import numpy as np

x_data = np.random.randn(2000,3)
w_real = [0.3,0.5,0.1]
b_real = -0.2

noise = np.random.randn(1,2000) * 0.1
y_data = np.matmul(w_real,x_data.T) + b_real + noise

(2000, 3)


## 선형 회귀

In [154]:
NUM_STEP = 10

g = tf.Graph()
wb_ = []
with g.as_default():
    x = tf.placeholder(tf.float32,shape=[None,3])
    y_true = tf.placeholder(tf.float32,shape=None)
    
    with tf.name_scope('inference') as scope:
        w = tf.Variable([[0,0,0]],dtype=tf.float32,name='weight')
        b = tf.Variable(0,dtype=tf.float32,name='bias')
        y_pred = tf.matmul(w,tf.transpose(x)) + b
        
    with tf.name_scope('loss') as scope:
        loss = tf.reduce_mean(tf.square(y_true-y_pred)) # square = 제곱
        
    with tf.name_scope('train') as scope:
        learning_rate = 0.4
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train = optimizer.minimize(loss)
        
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        for step in range(NUM_STEP+1):
            sess.run(train,{x:x_data,y_true:y_data})
            print(step, sess.run([w,b]))
            wb_.append(sess.run([w,b]))

0 [array([[0.23581381, 0.42544815, 0.08778735]], dtype=float32), -0.18028514]
1 [array([[0.28574127, 0.48811623, 0.09964821]], dtype=float32), -0.20130846]
2 [array([[0.29631522, 0.4976568 , 0.10058563]], dtype=float32), -0.20334913]
3 [array([[0.2985782 , 0.49917218, 0.1004322 ]], dtype=float32), -0.20343208]
4 [array([[0.29907048, 0.49942562, 0.10032216]], dtype=float32), -0.20339674]
5 [array([[0.29917958, 0.49947053, 0.10028174]], dtype=float32), -0.2033807]
6 [array([[0.29920426, 0.499479  , 0.10026936]], dtype=float32), -0.20337582]
7 [array([[0.29920992, 0.4994807 , 0.10026588]], dtype=float32), -0.2033745]
8 [array([[0.29921126, 0.49948105, 0.10026494]], dtype=float32), -0.20337416]
9 [array([[0.29921156, 0.4994811 , 0.10026469]], dtype=float32), -0.20337409]
10 [array([[0.29921165, 0.49948114, 0.10026463]], dtype=float32), -0.20337406]


## 로지스틱 회귀
- 시그모이드

In [155]:
N = 20000

def sigmoid(x):
    return 1./(1+np.exp(-x))

x_data = np.random.randn(N,3)
w_real = [0.3,0.5,0.1]
b_real = -0.2

wxb = np.matmul(w_real,x_data.T) + b_real

y_data_pre_noise = sigmoid(wxb)
y_data = np.random.binomial(1,y_data_pre_noise)

In [162]:

NUM_STEPS = 50


g = tf.Graph()
wb_ = []
with g.as_default():
    x = tf.placeholder(tf.float32,shape=[None,3])
    y_true = tf.placeholder(tf.float32,shape=None)
    
    with tf.name_scope('inference') as scope:
        w = tf.Variable([[0,0,0]],dtype=tf.float32,name='weights')
        b = tf.Variable(0,dtype=tf.float32,name='bias')
        y_pred = tf.matmul(w,tf.transpose(x)) + b

    with tf.name_scope('loss') as scope:
        loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true,logits=y_pred) 
        loss = tf.reduce_mean(loss)
  
    with tf.name_scope('train') as scope:
        learning_rate = 0.5
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train = optimizer.minimize(loss)



    # Before starting, initialize the variables.  We will 'run' this first.
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)      
        for step in range(NUM_STEPS):
            sess.run(train,{x: x_data, y_true: y_data})
            if (step % 5 == 0):
                print(step, sess.run([w,b]))
                wb_.append(sess.run([w,b]))

        print(50, sess.run([w,b]))
        

0 [array([[0.03300837, 0.05694366, 0.00933739]], dtype=float32), -0.02182497]
5 [array([[0.14626522, 0.2521679 , 0.04147508]], dtype=float32), -0.09694247]
10 [array([[0.2068761 , 0.35650367, 0.05875371]], dtype=float32), -0.13727812]
15 [array([[0.24071355, 0.41468257, 0.06843163]], dtype=float32), -0.1598067]
20 [array([[0.26014146, 0.4480508 , 0.07400179]], dtype=float32), -0.17272577]
25 [array([[0.27148488, 0.46751493, 0.07726028]], dtype=float32), -0.18025354]
30 [array([[0.27817377, 0.47898254, 0.07918473]], dtype=float32), -0.184682]
35 [array([[0.28214106, 0.4857789 , 0.08032767]], dtype=float32), -0.18730222]
40 [array([[0.2845022 , 0.48982093, 0.0810087 ]], dtype=float32), -0.18885799]
45 [array([[0.28591028, 0.4922299 , 0.08141523]], dtype=float32), -0.18978372]
50 [array([[0.28661558, 0.49343595, 0.08161904]], dtype=float32), -0.19024655]
